In [3]:
%matplotlib notebook
%autosave 0
%load_ext autoreload
%autoreload 2
#%matplotlib inline
import numpy as np
from numpy.random import RandomState
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import animation
#from os.path import join
#import ipywidgets as widgets
from IPython.display import display
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torch import nn, cuda
from torch.autograd import Variable
import pickle
#from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix, classification_report
from tqdm.notebook import tqdm

Autosave disabled
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append("../imbalanced_dataset_sampler/torchsampler/")
sys.path.append("../libs")
from vvv_utils import parse_metadata, parse_light_curve_data, plot_light_curve, get_train_test_ids
#from imbalanced import ImbalancedDatasetSampler


### Parse Metadata

In [6]:
df_meta = parse_metadata(experiment="ALL", merge_subclasses=True)
class_names = df_meta["label"].unique()
display(df_meta.head(5))

88454 light curve metadata collected


,P1,P2,P_ogle,label
ID_VVV,,,,
b221_201_22183,13.972335,6.986167,13.970992,binary
b221_205_41463,21.537799,10.770059,21.541892,binary
b232_614_24529,8.035356,16.069420,16.065114,binary
b233_201_30278,8.750438,4.374645,8.751487,binary
b233_201_16631,44.130627,22.070183,44.161985,binary


In [9]:
indices_vvv = np.arange(len(df_meta))
#indices_vvv

In [10]:
df_meta["idx"] = indices_vvv
df_meta = df_meta.reset_index()
df_meta

,ID_VVV,P1,P2,P_ogle,label,idx
0,b221_201_22183,13.972335,6.986167,13.970992,binary,0
1,b221_205_41463,21.537799,10.770059,21.541892,binary,1
2,b232_614_24529,8.035356,16.069420,16.065114,binary,2
3,b233_201_30278,8.750438,4.374645,8.751487,binary,3
4,b233_201_16631,44.130627,22.070183,44.161985,binary,4
...,...,...,...,...,...,...
88449,b395_201_28153,0.316792,0.633585,0.316787,rrlyrae,88449
88450,b395_205_6265,0.287350,0.574699,0.287350,rrlyrae,88450
88451,b395_304_9934,0.355847,0.711693,0.355842,rrlyrae,88451
88452,b396_209_39823,0.814810,0.407405,0.407412,rrlyrae,88452


In [11]:
#df_meta = df_meta[["idx","ID_VVV","P1","P2","P_ogle","label"]]
df_meta = df_meta.set_index(df_meta["idx"])
df_meta


,ID_VVV,P1,P2,P_ogle,label,idx
idx,,,,,,
0,b221_201_22183,13.972335,6.986167,13.970992,binary,0
1,b221_205_41463,21.537799,10.770059,21.541892,binary,1
2,b232_614_24529,8.035356,16.069420,16.065114,binary,2
3,b233_201_30278,8.750438,4.374645,8.751487,binary,3
4,b233_201_16631,44.130627,22.070183,44.161985,binary,4
...,...,...,...,...,...,...
88449,b395_201_28153,0.316792,0.633585,0.316787,rrlyrae,88449
88450,b395_205_6265,0.287350,0.574699,0.287350,rrlyrae,88450
88451,b395_304_9934,0.355847,0.711693,0.355842,rrlyrae,88451


### Definiciones de Clases, Variables, Datasets

In [17]:
lc_classes = {"label":{"binary":0, "rrlyrae":1, "cepheid":2}}

In [18]:
# Variable Categórica, reemplaza nombres por números
df_meta.replace(lc_classes,inplace=True)
df_meta

,ID_VVV,P1,P2,P_ogle,label,idx
idx,,,,,,
0,b221_201_22183,13.972335,6.986167,13.970992,0,0
1,b221_205_41463,21.537799,10.770059,21.541892,0,1
2,b232_614_24529,8.035356,16.069420,16.065114,0,2
3,b233_201_30278,8.750438,4.374645,8.751487,0,3
4,b233_201_16631,44.130627,22.070183,44.161985,0,4
...,...,...,...,...,...,...
88449,b395_201_28153,0.316792,0.633585,0.316787,1,88449
88450,b395_205_6265,0.287350,0.574699,0.287350,1,88450
88451,b395_304_9934,0.355847,0.711693,0.355842,1,88451


In [87]:
# NUEVO

class LightCurve_Dataset(Dataset):
    def __init__(self, df_metadata, transform=None):        
        self.data = list()
        self.name = df_metadata["ID_VVV"]
        self.period = df_metadata["P_ogle"]
        self.label = torch.from_numpy(df_metadata["label"].values)
        self.transform = transform
        column_names_lc = ["mjd", "mag", "err"]
        column_names_fill = ["mjd", "mag", "err", "phase", "mask"]
        df_zeros = pd.DataFrame(np.zeros((1, 5)),columns=column_names_fill)
        
        for i in range(len(df_metadata)):
            lc_data = parse_light_curve_data(self.name[i])
            lc_data["phase"] = np.mod(lc_data["mjd"],self.period[i])/self.period[i]
            #display(lc_data)
            #normalize
            mag_std = lc_data["mag"].std()
            lc_data["mjd"] = lc_data["mjd"]-lc_data["mjd"].min()
            lc_data["mag"] = (lc_data["mag"]-lc_data["mag"].mean())/mag_std
            lc_data["err"] = lc_data["err"]/mag_std
            lc_data.sort_values(by="phase", inplace=True)
            # ajustar todas a largo 335, rellenando con 0s las que sean mas pequeñas,
            # asignando un label '1' si es dato real y '0' si es dato rellenado.
            """if len(lc_data) == 335:
                lc_data["mask"] = 1
            else:
                while len(lc_data) < 335:
                    #RELLENAR con 0s;
                    lc_data = lc_data.append(df_zeros, ignore_index=True,sort=False)
            lc_data = lc_data.fillna(1)
            lc_data = lc_data[["phase","mag","err","mjd","mask"]]
            self.data.append(torch.from_numpy(lc_data.values.astype('float32')))
            """
            lc_data = lc_data[["phase","mag","err","mjd"]]
            lc_data_large = np.zeros(shape=(335, 5), dtype='float32')
            lc_data_large[:lc_data.shape[0], :4] = lc_data.values
            lc_data_large[:lc_data.shape[0], 4] = 1.
            self.data.append(torch.from_numpy(lc_data_large))
            
    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.label[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample

    def plot(self, idx, ax):
        assert len(ax)==2, "Needs two subaxis"
        ax[0].cla()  
        ax[0].errorbar(self.data[idx][:, 0], self.data[idx][:, 1], self.data[idx][:, 2], fmt='.')
        ax[0].invert_yaxis()
        ax[1].cla()
        ax[1].errorbar(self.data[idx][:, 3], self.data[idx][:, 1], self.data[idx][:, 2], fmt='.')
        ax[1].invert_yaxis()
        ax[0].set_title("%d %s %0.4f" %(idx, self.name[idx],self.period[idx]))

    def __len__(self):
        return len(self.data)

**Dataset**

In [ ]:
df_meta

In [ ]:
%%timeit -n 1 -r 1
#para evaluar el tiempo

In [ ]:
#%%timeit -n 1 -r 1
# CUIDADO, el parseo demora más de 1 hora y 45 min (me aburri de esperar). 
VVV = LightCurve_Dataset(df_meta)

In [ ]:
VVV[420]

In [ ]:
# próxima ocasión CARGAR dataset desde pickle
with open('/home/amorales/tesis/LC_test/pickles/newDataset_VVV.pkl', 'rb') as handle:
    VVV = pickle.load(handle)


Guardar Dataset como Pickle

In [ ]:
with open('/home/amorales/tesis/LC_test/pickles/newDataset_VVV.pkl', 'wb') as handle:
    pickle.dump(VVV, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
fig,ax=plt.subplots(2,1)
VVV.plot(80000,ax)

In [ ]:
lc_max=0
lc_min=np.inf
for i in VVV:
    if len(i['data'])>lc_max:
        lc_max=len(i['data'])
    if len(i['data'])<lc_min:
        lc_min=len(i['data'])
print(lc_min,lc_max)

In [ ]:
VVV[420]['label']

In [ ]:
#phase = VVV[i]['data'][:,0]
#mag = VVV[i]['data'][:,1]
#err = VVV[i]['data'][:,2]
#mask = VVV[i]['data'][:,4]
#label = VVV[i]['label'].item()
    # 0: Binary
    # 1: RRL
    # 2: Cepheid

In [ ]:
VVV_len = Counter(VVV[:]['label'].numpy())

In [ ]:
n_Bin = (VVV[:]['label'].numpy() == 0).sum()
n_RRL = (VVV[:]['label'].numpy() == 1).sum()
n_Cep = (VVV[:]['label'].numpy() == 2).sum()
VVV_len = np.array([n_Bin, n_RRL, n_Cep])
VVV_len

Generar Sampleo Random Manual Equitativo (EMRS)

In [ ]:
class equal_manually_random(Dataset):    
    def __init__(self, data, targets,transform=None):
        assert torch.is_tensor(data)==True and torch.is_tensor(targets)==True
        self.data=data
        self.labels=targets
        self.transform=transform
    
    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return len(self.data)

In [ ]:
def MRS_dataset(rs):
    # Sampleo Random Manual NoEquitativo (NEMRS)
    less_lc = VVV_len.min()
    VVV_toy = list()
    ant=0
    for i,n_lc in enumerate(VVV_len):
        if i==0:
            rand_ind = np.random.permutation(np.arange(n_lc))[:less_lc*3]+ant
        elif i==1:
            rand_ind = np.random.permutation(np.arange(n_lc))[:less_lc*2]+ant
        elif i==2:
            rand_ind = np.random.permutation(np.arange(n_lc))[:less_lc]+ant
        VVV_toy.append(rand_ind)
        ant += n_lc
    VVV_sameRandInd = np.concatenate(VVV_toy)
    # Data filter
    targets = [VVV[i]['label'].item() for i in VVV_sameRandInd]
    targets = torch.tensor(targets,dtype=torch.long)
    LC_magData = [VVV[i]['data'][:,1].numpy() for i in VVV_sameRandInd]
    LC_errData = [VVV[i]['data'][:,2].numpy() for i in VVV_sameRandInd]
    LC_mask = [VVV[i]['data'][:,4].numpy() for i in VVV_sameRandInd]
    LC_magData = torch.tensor(LC_magData)
    LC_errData = torch.tensor(LC_errData)
    LC_mask = torch.tensor(LC_mask)
    mag_err_mask=torch.stack((LC_magData,LC_errData,LC_mask),dim=1)
    # Dataset
    VVV_equalDataset = equal_manually_random(mag_err_mask, targets)
    return VVV_equalDataset

### Dataloaders

In [ ]:
train_idx, test_idx = get_train_test_ids(df_meta)

from collections import Counter

display(Counter(list(df_meta.loc[train_idx]["label"])))
display(Counter(list(df_meta.loc[test_idx]["label"])))

In [ ]:
train_idx

In [ ]:
def make_dataloaders(dataset,use_IDS,split,train_size=32,test_size=64):
    global train_idx
    global test_idx
    if use_IDS:
        # ImbalancedDatasetSampler
        #train_idx, valid_idx= train_test_split(
        #                                np.arange(len(dataset)),
        #                                test_size=split,
        #                                shuffle=True,
        #                                stratify=dataset[:]['label'])
        #train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(dataset,
                          sampler=ImbalancedDatasetSampler(dataset,
                                                           indices=train_idx,
                                                           callback_get_label= lambda dataset, idx:dataset[idx]['label'].item()),
                          batch_size=train_size, shuffle=False)

        test_loader= DataLoader(dataset,
                                sampler=SubsetRandomSampler(test_idx),
                                batch_size=test_size, shuffle=False)
    else:
        train_idx, valid_idx= train_test_split(
                                        np.arange(len(dataset)),
                                        test_size=split,
                                        shuffle=True,
                                        stratify=None)
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(dataset,
                                sampler=SubsetRandomSampler(train_idx),
                                batch_size=train_size, shuffle=False)

        test_loader = DataLoader(dataset,
                                sampler=SubsetRandomSampler(valid_idx),
                                batch_size=test_size, shuffle=False)
    return train_loader, test_loader

### Def. Modelo NN

In [ ]:
# implementacion adaptada a 1D de https://github.com/naoto0804/pytorch-inpainting-with-partial-conv

class PartialConv(nn.Module):
    def __init__(self, in_channels_C,in_channels_M, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super().__init__()
        self.input_conv = nn.Conv1d(in_channels_C, out_channels, kernel_size,
                                    stride, padding, dilation, groups, bias)
        self.mask_conv = nn.Conv1d(in_channels_M, out_channels, kernel_size,
                                   stride, padding, dilation, groups, False)
        # self.input_conv.apply(weights_init('kaiming'))

        torch.nn.init.constant_(self.mask_conv.weight, 1.0)

        # mask is not updated
        for param in self.mask_conv.parameters():
            param.requires_grad = False

    def forward(self,input, mask):
        # http://masc.cs.gmu.edu/wiki/partialconv
        # C(X) = W^T * X + b, C(0) = b, D(M) = 1 * M + 0 = sum(M)
        # W^T* (M .* X) / sum(M) + b = [C(M .* X) – C(0)] / D(M) + C(0)
        #print(input.shape, mask.shape)
        output = self.input_conv(input * mask)
        if self.input_conv.bias is not None:
            output_bias = self.input_conv.bias.view(1, -1, 1).expand_as(output)
        else:
            output_bias = torch.zeros_like(output)

        with torch.no_grad():
            output_mask = self.mask_conv(mask)

        no_update_holes = output_mask == 0
        mask_sum = output_mask.masked_fill_(no_update_holes, 1.0)

        output_pre = (output - output_bias) / mask_sum + output_bias
        output = output_pre.masked_fill_(no_update_holes, 0.0)

        new_mask = torch.ones_like(output)
        new_mask = new_mask.masked_fill_(no_update_holes, 0.0)

        return output, new_mask

In [ ]:
class MLP_gap(torch.nn.Module):
    def __init__(self, in_channels_C=2,in_channels_M=1,c1=128,c2=64, c3=32, c4=16,c5=8, kernel_size=3, hid_dim=32,hid2_dim=16,hid3_dim=8,output_dim=5): 
        super(MLP_gap, self).__init__()
        self.pconv1 = PartialConv(in_channels_C,in_channels_M, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        self.pconv2 = PartialConv(c4, c4, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        self.pconv3 = PartialConv(c4, c4, c2, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        #self.pconv4 = PartialConv(c4, c4, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        #self.pool1 = torch.nn.AvgPool1d(kernel_size, stride=None, padding=0,
        #                                 ceil_mode=False, count_include_pad=True)
        self.gap = torch.nn.AdaptiveMaxPool1d(1)
        
        self.hidden1 = torch.nn.Linear(c2*1, hid_dim, bias=True)
        self.hidden2 = torch.nn.Linear(hid_dim, hid2_dim, bias=True)
        #self.hidden3 = torch.nn.Linear(hid2_dim, hid3_dim, bias=True)
        self.output = torch.nn.Linear(hid2_dim, output_dim, bias=True)

        self.activation = torch.nn.ReLU()
        #self.soft = torch.nn.Softmax(dim=1)
        
    def forward(self, x,mask, t=0):
        if t==1:
            x = x.transpose(0,1)
            mask = mask.transpose(0,1)
        elif t==2:
            #x = x.transpose(1,2)
            mask = mask.transpose(1,2)
    
        x, mask = self.pconv1(x, mask)
        x = self.activation(x)
        x, mask = self.pconv2(x, mask)
        x = self.activation(x)
        x, mask = self.pconv3(x, mask)
        x = self.activation(x)
        #x, mask = self.pconv4(x, mask)
        z = self.gap(x)
        z = z.reshape(-1,64*1)
        z = self.activation(self.hidden1(z))
        z = self.activation(self.hidden2(z))
        #z = self.activation(self.hidden3(z))
        fuera = self.output(z)
        return fuera


In [ ]:
class MLP_last(torch.nn.Module):
    def __init__(self, in_channels_C=2,in_channels_M=1,c1=128,c2=64, c3=32, c4=16,c5=8, kernel_size=3, hid_dim=32,hid2_dim=16,hid3_dim=8,output_dim=3): 
        super(MLP_last, self).__init__()
        self.pconv1 = PartialConv(in_channels_C,in_channels_M, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        self.pconv2 = PartialConv(c4, c4, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        self.pconv3 = PartialConv(c4, c4, c2, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        #self.pconv4 = PartialConv(c4, c4, c4, kernel_size, stride=2, padding=0, dilation=1, bias=True)
        #self.pool1 = torch.nn.AvgPool1d(kernel_size, stride=None, padding=0,
        #                                 ceil_mode=False, count_include_pad=True)
        self.gap = torch.nn.AdaptiveMaxPool1d(1)
        
        self.hidden1 = torch.nn.Linear(c2*1, hid_dim, bias=True)
        self.hidden2 = torch.nn.Linear(hid_dim, hid2_dim, bias=True)
        #self.hidden3 = torch.nn.Linear(hid2_dim, hid3_dim, bias=True)
        self.output = torch.nn.Linear(hid2_dim, output_dim, bias=True)

        self.activation = torch.nn.ReLU()
        #self.soft = torch.nn.Softmax(dim=1)
        
    def forward(self, x,mask, t=0):
        if t==1:
            x = x.transpose(0,1)
            mask = mask.transpose(0,1)
        elif t==2:
            #x = x.transpose(1,2)
            mask = mask.transpose(1,2)
    
        x, mask = self.pconv1(x, mask)
        x = self.activation(x)
        x, mask = self.pconv2(x, mask)
        x = self.activation(x)
        x, mask = self.pconv3(x, mask)
        x = self.activation(x)
        #x, mask = self.pconv4(x, mask)
        z = self.gap(x)
        z = z.reshape(-1,64*1)
        z = self.activation(self.hidden1(z))
        z = self.activation(self.hidden2(z))
        #z = self.activation(self.hidden3(z))
        fuera = self.output(z)
        return fuera



In [ ]:
modelo = MLP_last()
display(modelo)

In [ ]:
total_params = sum(p.numel() for p in modelo.parameters() if p.requires_grad)
total_params

#### BetaTesting-b4-Train

**Check GPU**

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.cuda.set_device('cuda:2')

### ENTRENAMIENTO

In [ ]:
def update_plot(ax,TL,VL,F1,last_ep,best_f1,topf1_logs):
    #global comentario
    titulo="Current Training: Sample="+str(s)
    [ax_.cla() for ax_ in ax]
    ax[0].plot(range(len(TL)), TL, lw=2, label='Train')
    ax[0].plot(range(len(VL)), VL, lw=2, label='Valid')
    ax[0].axvline(last_ep,c='r',marker='|',lw=1,label="Last improve")
    ax[0].plot(range(len(VL)),np.full_like(VL,min(VL)),'r--',lw=1, label='best VL')
    ax[1].plot(range(len(F1)), F1, lw=2, label='F1 score')
    if len(topf1_logs)==0:
        # plot topf1 actual
        ax[1].plot(range(len(F1)),np.full_like(F1,best_f1),'r--',lw=1, label='top F1')
        
    else:
        # plot topf1 mean
        ax[1].plot(range(len(F1)),np.full_like(F1,best_f1),'r--',lw=1, label='top F1')
        ax[1].plot(range(len(F1)),np.full_like(F1,np.mean(topf1_logs)),'g--',lw=1, label='mean F1')
    [ax_.set_xlabel('Epochs') for ax_ in ax]
    [ax_.grid() for ax_ in ax]
    [ax_.legend() for ax_ in ax]
    fig.suptitle(titulo)
    fig.canvas.draw()
    fig.canvas.flush_events()

In [ ]:
#phase = VVV[i]['data'][:,0]
#mag = VVV[i]['data'][:,1]
#err = VVV[i]['data'][:,2]
#mask = VVV[i]['data'][:,4]
#label = VVV[i]['label'].item()
    # 0: Binary
    # 1: RRL
    # 2: Cepheid

In [ ]:
def training(modelo, criterion, lr,wd, epochs, use_gpu, train_loader, test_loader,path,ax,topf1_logs):
    optimizer = torch.optim.Adam(modelo.parameters(), lr=lr,weight_decay=wd)
    ultima_mejora = 0
    best_valid,best_f1= np.inf,0.0
    TL,VL,F1=list(),list(),list()

    for k in epochs:
        if k-ultima_mejora >=300:
            print("Hace al menos 300 épocas NO disminuye Valid Loss. Best F1=",best_f1)
            break
        train_loss, valid_loss,f1_acum = 0.0, 0.0, 0.0
        # Loop de entrenamiento
        for batch in train_loader:
            inputs, labels = batch['data'],batch['label']
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.transpose(1,2)
            #display(inputs.shape,labels.shape)
            #break
            data,mask = inputs[:,[1,2]],inputs[:,4].unsqueeze(0).transpose(0,1)
            #display(data.shape,mask.shape)
            #display(data,mask)
            optimizer.zero_grad()
            outputs = modelo.forward(data,mask,t=0)
            #display(outputs.shape,labels.shape)
            #break
            loss = criterion(outputs,labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        TL.append(train_loss/train_loader.__len__()) 

        # VALIDACION 
        for batch in test_loader:
            inputs, labels = batch['data'],batch['label']
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.transpose(1,2)
            #display(inputs.shape,labels.shape)
            data,mask = inputs[:,[1,2]],inputs[:,4].unsqueeze(0).transpose(0,1)
            
            y_true=labels.cpu().numpy()
            #display(y_true)
            #break
            outputs = modelo.forward(data,mask,t=0)
            # F1 score
            y_pred=outputs.cpu().detach().argmax(dim=1).numpy()
            #print(y_pred)
            f1_acum += f1_score(y_true, y_pred, average='weighted')
            loss = criterion(outputs,labels)
            #print(loss.item())
            valid_loss += loss.item()
            #print(valid_loss)
            # save best model
        if valid_loss < best_valid:
            ultima_mejora = k+1
            #print("vamos mejorando :) epoch=",k+1)
            best_valid = valid_loss
            best_f1 = f1_acum/test_loader.__len__()
            torch.save({'epoca': k,
                        'f1_score': best_f1,
                        'model_state_dict': modelo.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'Valid_loss': valid_loss/test_loader.__len__()}, path)
        VL.append(valid_loss/test_loader.__len__())
        F1.append(f1_acum/test_loader.__len__())
        update_plot(ax,TL,VL,F1,ultima_mejora,best_f1,topf1_logs)

    # Retornar modelo a la CPU
    if use_gpu:
        modelo = modelo.cpu()
        #print("ok")
    return best_f1

In [ ]:
# Parametros generales
n_epochs=10000
lr=0.0002
wd = 0
epochs = range(n_epochs)
use_gpu = True
use_IDS = True
criterion = torch.nn.CrossEntropyLoss()
semilla=774892200
rs=RandomState(semilla)
split,train_size,test_size=0.3,32,64

In [ ]:
# New Dataset
fig, ax = plt.subplots(1, 2, figsize=(8.5, 4), tight_layout=True, sharex=True)
top5f1=list()
prompt="Training new Dataset"
print("\nBegin",prompt)
for s in range(5): 
    comentario = "_sample"+str(s)
    path = '/home/amorales/tesis/LC_test/models/LCC_newDataset_model_01_'+comentario+'.pt'
    # DataLoaders
    train_loader, test_loader = make_dataloaders(VVV,use_IDS,split,train_size,test_size)
    # ENTRENAR
    modelo=MLP_last()
    if use_gpu:
        modelo=modelo.cuda()
    topf1 = training(modelo, criterion, lr,wd, epochs, use_gpu, train_loader, test_loader,path,ax,top5f1)
    top5f1.append(topf1)

# return f1 mean, f1 std
print("Results: F1 mean=",np.mean(top5f1),"F1 std=",np.std(top5f1))
print("End",prompt)
print("-------END-------")

In [ ]:
# original práctica
fig, ax = plt.subplots(1, 2, figsize=(8.5, 4), tight_layout=True, sharex=True)
# for IDS in [0,1]:
for rot_key in [0,1]:
    for alpha in [0,0.25,0.5,0.75,1]:
        top5f1=list()
        prompt="Training. Rot="+str(rot_key)+", Alpha="+str(alpha)
        print("\nBegin",prompt)
        for s in range(5): 
            comentario = "_Rot"+str(rot_key)+"_Alpha"+str(alpha)+"_sampleo"+str(s)
            path = '/home/amorales/models/best_LCC_model_last_'+comentario+'.pt'
            # sampleo random manual y crear dataset
            VVV_eqData=MRS_dataset(rs)
            # DataLoaders
            train_loader, test_loader = make_dataloaders(VVV_eqData,use_IDS,split,train_size,test_size)
            # ENTRENAR
            modelo=MLP_last()
            if use_gpu:
                modelo=modelo.cuda()
            topf1 = training(modelo, criterion, lr,wd, epochs, use_gpu, train_loader, test_loader,path,ax,top5f1)
            top5f1.append(topf1)
        
        # return f1 mean, f1 std
        print("Results: F1 mean=",np.mean(top5f1),"F1 std=",np.std(top5f1))
        print("End",prompt)
print("-------END-------")

**Matriz de Confusión**

In [ ]:
besto = MLP()
PATH = '/home/amorales/tesis/models/best_LCC_model_dataRot0_Alpha1_sampleo4.pt'
besto.load_state_dict(torch.load(PATH)['model_state_dict'])

In [ ]:
#Junio 2020
besto = MLP_last()
PATH = '/home/amorales/tesis/models/best_LCC_model_dataRot0_Alpha1_sampleo4.pt'
besto.load_state_dict(torch.load(path)['model_state_dict'])

In [ ]:
# MODIFICADA New Dataset ()
#y_true = [best_of_best[i]['label'].item() for i in range(len(best_of_best))]
#y_true = [VVV[i]['label'].item() for i in valid_idx]

prediction_test = []
labels_test=[]
for batch in test_loader:
    inputs, labels = batch['data'],batch['label']
    inputs = inputs.transpose(1,2)
    #display(inputs.shape,labels.shape)
    data,mask = inputs[:,[1,2]],inputs[:,4].unsqueeze(0).transpose(0,1)
    labels_test.append(labels.detach().numpy())
    #y_true=labels.cpu().numpy()
    #display(y_true)
    #break
    outputs = besto.forward(data,mask,t=0)
    # F1 score
    y_pred=outputs.detach().argmax(dim=1).numpy()
    #print(outputs)
    prediction_test.append(outputs.detach().argmax(dim=1).numpy())
y_true = np.concatenate(labels_test)
y_pred = np.concatenate(prediction_test)
print(y_pred.__len__()/5)
cm = confusion_matrix(y_true, y_pred)
display(cm)

print(classification_report(y_true, y_pred))

In [ ]:
# MODIFICAR
#y_true = [best_of_best[i]['label'].item() for i in range(len(best_of_best))]
#y_true = [VVV[i]['label'].item() for i in valid_idx]

prediction_test = []
labels_test=[]
for data in test_loader:
    inputs, labels = data['data'],data['label']
    data,mask = inputs[:,:2],inputs[:,2].unsqueeze(-1)
    labels_test.append(labels.detach().numpy())
    outputs = besto.forward(data,mask,t=2)
    #logits = modelo.forward(sample_data)
    #print(outputs)
    prediction_test.append(outputs.detach().argmax(dim=1).numpy())
y_true = np.concatenate(labels_test)
y_pred = np.concatenate(prediction_test)
print(y_pred.__len__()/5)
cm = confusion_matrix(y_true, y_pred)
display(cm)

print(classification_report(y_true, y_pred))

### Comparación entre distintos Modelos
Historial Matrices de Confusion [https://docs.google.com/spreadsheets/d/1j9kseGSx1WQvkb1C4nDL5J-lweuH3VyPsrAT50E1D6w/edit#gid=0]

In [ ]:
with open('lc_debug.pkl', 'rb') as handle:
    lc_debug = pickle.load(handle)

In [ ]:
path1= '/home/amorales/models/best_LCC_model_NoPool__Rot1_Alpha1_sampleo1.pt'
path0= '/home/amorales/models/best_LCC_model__Rot1_Alpha1_sampleo0.pt'

In [ ]:
besto0,besto1 = MLP_(),MLP()
#PATH = '/home/amorales/models/best_LCC_model_dataRot0_Alpha1_sampleo4.pt'
besto0.load_state_dict(torch.load(path0)['model_state_dict'])
besto1.load_state_dict(torch.load(path1)['model_state_dict'])

In [ ]:
pond=2.3
fig1, ax1 = plt.subplots(6, 4, figsize=(4*pond, 6*pond), tight_layout=True,sharey=True,sharex=False)
fig2, ax2 = plt.subplots(6, 4, figsize=(4*pond, 6*pond), tight_layout=True,sharey=True,sharex=True)
fig1.suptitle("Modelo sin Activacion")
fig2.suptitle("Modelo con Activacion, sin Pooling")
for i,lc in enumerate(lc_debug):
    data,label=lc['data'],lc['label']
    n = sum(data[:,4]==1).item()
    print(n)
    pha = data[:,0]
    mag = data[:,1]
    err = data[:,2]
    data2=torch.stack((mag,err))
    data,mask = data2.unsqueeze(0),data[:,4].unsqueeze(-1).unsqueeze(0).transpose(1,2)
    #data,mask = data[:,:2].unsqueeze(0).transpose(1,2),data[:,4].unsqueeze(0).unsqueeze(-1).transpose(1,2)
    norm = Normalize(0,1,clip=False)
    for j in range(4):
        if i<3:
            star="ECL"
        else:
            star="RRL"
        if j!=3:
            layer="Conv "+str(j+1)
        else:
            layer="GAP"
        if j==0:
            data0,mask0=besto0.pconv1(data,mask)
            data1,mask1=besto1.pconv1(data,mask)
            data1=besto1.activation(data1)

        elif j==1:
            data0,mask0=besto0.pconv2(data0,mask0)
            data1,mask1=besto1.pconv2(data1,mask1)
            data1=besto1.activation(data1)

        elif j==2:
            data0,mask0=besto0.pconv3(data0,mask0)
            data1,mask1=besto1.pconv3(data1,mask1)
            data1=besto1.activation(data1)

        else:       
            mask0_tmp=mask0.squeeze()[0].numpy().astype('int32')
            mask1_tmp=mask1.squeeze()[0].numpy().astype('int32')
            m0,m1= sum(mask0_tmp),sum(mask1_tmp)
            print(m0,m1)
            data0_tmp=data0.squeeze().detach()
            data1_tmp=data1.squeeze().detach()
            data0_slice=data0_tmp[:,:m0].unsqueeze(0)
            data1_slice=data1_tmp[:,:m1].unsqueeze(0)
            """mask0_tmp=mask0.bool()
            data0_slice=data0[mask0_tmp]
            mask1_tmp=mask1.bool()
            data1_slice=data1[mask1_tmp]
            display(mask1_tmp.shape,data1_slice.shape)"""
            data0=besto0.gap(data0_slice)
            data1=besto1.gap(data1_slice)
            display(data0.shape,data1.shape)
        #plot
        #ax1[i][j].errorbar(pha,mag,err, fmt='b.')
        #ax2[i][j].errorbar(pha,mag,err, fmt='b.')
        filtro0=data0.squeeze().detach().numpy()
        filtro1=data1.squeeze().detach().numpy()
        ax1[i][j].imshow(filtro0,norm=norm,cmap='plasma')
        ax2[i][j].imshow(filtro1,norm=None)
        ax1[i][j].set_title(star+" after "+layer+", n= "+str(n))
        ax2[i][j].set_title(star+" after "+layer+", n= "+str(n))
        if i==0 and j==0:
            ax1[i][j].invert_yaxis()
            ax2[i][j].invert_yaxis()

        

In [ ]:
torch.repeat_interleave?

In [ ]:
x = torch.tensor([[1,2,3],[4,5,6]])
y = torch.from_numpy(np.random.random(100))
y_rp = y.repeat_interleave(2,dim=0)
display(y,y_rp,y.shape,y_rp.shape)

In [ ]:
pond=2
#fig, ax = plt.subplots(4, 2, figsize=(4*pond, 6*pond), tight_layout=False)
fig, ax = plt.subplots(1, 2, figsize=(8.5, 4), tight_layout=True, sharex=False)
data=lc_debug[1]['data']
n = sum(data[:,4]==1).item()
print(n)
mag = data[:,1]
err = data[:,2]
data2=torch.stack((mag,err))
data,mask = data2.unsqueeze(0),data[:,4].unsqueeze(-1).unsqueeze(0).transpose(1,2)
data1,mask1=besto0.pconv1(data,mask)
mask_tmp=mask1.squeeze()[0].numpy().astype('int32')
m= sum(mask_tmp)
print(m)
data_tmp=data1.squeeze().detach()
data_slice=data_tmp[:,:m]
data_repeat=data_slice.repeat_interleave(2,dim=1)[:,:-1]
display(data_repeat.shape,data_slice.shape)
ax[0].imshow(data_repeat)
ax[1].imshow(data_slice)